# Check Python version

In [ ]:
import sys
sys.version
import time
nb_start = time.time()

# Install PyTorch

In [2]:
!pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl 

    100% |████████████████████████████████| 592.3MB 29.2MB/s 


In [3]:
!pip3 install torchvision

    100% |████████████████████████████████| 61kB 2.7MB/s 
    100% |████████████████████████████████| 2.0MB 9.3MB/s 
  Found existing installation: Pillow 4.0.0
    Uninstalling Pillow-4.0.0:
      Successfully uninstalled Pillow-4.0.0


# Import PyTorch

In [ ]:
import torch
import torch.nn as nn

import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable

# Initialize Hyper-parameters

In [ ]:
input_size    = 784   # The image size = 28 x 28 = 784
hidden_size   = 500   # The number of nodes at the hidden layer
num_classes   = 10    # The number of output classes. In this case, from 0 to 9
num_epochs    = 5     # The number of times entire dataset is trained
batch_size    = 100   # The size of input data took for one iteration
learning_rate = 1e-3  # The speed of convergence

# Download MNIST Dataset

MNIST is a huge database of handwritten digits (i.e. 0 to 9) that is often used in image classification.

In [8]:
train_dataset = dsets.MNIST(root='./data',
                           train=True,
                           transform=transforms.ToTensor(),
                            
                           download=True)

test_dataset = dsets.MNIST(root='./data',
                           train=False,
                           transform=transforms.ToTensor())

Processing...
Done!


In [13]:
len(train_dataset.)

60000

# Load the Dataset

**Note**: We shuffle the loading process of `train_dataset` to make the learning process independent of data order, but the order of `test_loader` remains so as to examine whether we can handle unspecified bias order of inputs.


In [ ]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                          batch_size=batch_size,
                                          shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                          batch_size=batch_size,
                                          shuffle=False)

# Build the Feedforward Neural Network

### Feedforward Neural Network Model Structure

The FNN includes two fully-connected layers (i.e. fc1 & fc2) and a non-linear ReLU layer in between. Normally we call this structure **1-hidden layer FNN**, without counting the output layer (fc2) in.

By running the forward pass, the input images (x) can go through the neural network and generate a output (out) demonstrating how are the likabilities it belongs to each of the 10 classes. _For example, a cat image can have 0.8 likability to a dog class and a 0.3 likability to a airplane class._

In [ ]:
class Net(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(Net, self).__init__()                    # Inherited from the parent class nn.Module
        self.fc1 = nn.Linear(input_size, hidden_size)  # 1st Full-Connected Layer: 784 (input data) -> 500 (hidden node)
        self.relu = nn.ReLU()                          # Non-Linear ReLU Layer: max(0,x)
        self.fc2 = nn.Linear(hidden_size, num_classes) # 2nd Full-Connected Layer: 500 (hidden node) -> 10 (output class)
    
    def forward(self, x):                              # Forward pass: stacking each layer together
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

# Instantiate the FNN

We now create a real FNN based on our structure.

In [ ]:
net = Net(input_size, hidden_size, num_classes)

# Enable GPU

_**Note**: You could enable this line to run the codes on GPU_

In [ ]:
use_cuda = True

In [ ]:
if use_cuda and torch.cuda.is_available():
    net.cuda()
else:
  print("failed")

# Choose the Loss Function and Optimizer

Loss function (**criterion**) decides how the output can be compared to a class, which determines how good or bad the neural network performs. And the **optimizer** chooses a way to update the weight in order to converge to find the best weights in this neural network.

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(net.parameters(), lr=10e-4)

# Training the FNN Model

This process might take around 3 to 5 minutes depending on your machine. The detailed explanations are listed as comments (#) in the following codes.

In [27]:
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):   # Load a batch of images with its (index, data, class)
        images = Variable(images.view(-1, 28*28))         # Convert torch tensor to Variable: change image from a vector of size 784 to a matrix of 28 x 28
        labels = Variable(labels)
        
        if use_cuda and torch.cuda.is_available():
            images = images.cuda()
            labels = labels.cuda()
        
        optimizer.zero_grad()                             # Intialize the hidden weight to all zeros
        outputs = net(images)                             # Forward pass: compute the output class given a image
        loss = criterion(outputs, labels)                 # Compute the loss: difference between the output class and the pre-given label
        loss.backward()                                   # Backward pass: compute the weight
        optimizer.step()                                  # Optimizer: update the weights of hidden nodes
        
        if (i+1) % 100 == 0:                              # Logging
            print('Epoch [%d/%d], Step [%d/%d], Loss: %.4f'
                 %(epoch+1, num_epochs, i+1, len(train_dataset)//batch_size, loss.data[0]))


Epoch [1/5], Step [100/600], Loss: 0.0054
Epoch [1/5], Step [200/600], Loss: 0.0026
Epoch [1/5], Step [300/600], Loss: 0.0022
Epoch [1/5], Step [400/600], Loss: 0.0055
Epoch [1/5], Step [500/600], Loss: 0.0316
Epoch [1/5], Step [600/600], Loss: 0.0015
Epoch [2/5], Step [100/600], Loss: 0.0056
Epoch [2/5], Step [200/600], Loss: 0.0009
Epoch [2/5], Step [300/600], Loss: 0.0009
Epoch [2/5], Step [400/600], Loss: 0.0029
Epoch [2/5], Step [500/600], Loss: 0.0015
Epoch [2/5], Step [600/600], Loss: 0.0026
Epoch [3/5], Step [100/600], Loss: 0.0027
Epoch [3/5], Step [200/600], Loss: 0.0015
Epoch [3/5], Step [300/600], Loss: 0.0016
Epoch [3/5], Step [400/600], Loss: 0.0009
Epoch [3/5], Step [500/600], Loss: 0.0028
Epoch [3/5], Step [600/600], Loss: 0.0055
Epoch [4/5], Step [100/600], Loss: 0.0010
Epoch [4/5], Step [200/600], Loss: 0.0043
Epoch [4/5], Step [300/600], Loss: 0.0115
Epoch [4/5], Step [400/600], Loss: 0.0505
Epoch [4/5], Step [500/600], Loss: 0.0013
Epoch [4/5], Step [600/600], Loss:

# Testing the FNN Model

Similar to training the neural network, we also need to load batches of test images and collect the outputs. The differences are that:

1. No loss & weights calculation
2. No wights update
3. Has correct prediction calculation


In [28]:
correct = 0
total = 0
for images, labels in test_loader:
    images = Variable(images.view(-1, 28*28))
    
    if use_cuda and torch.cuda.is_available():
        images = images.cuda()
        labels = labels.cuda()
    
    
    outputs = net(images)
    _, predicted = torch.max(outputs.data, 1)  # Choose the best class from the output: The class with the best score
    total += labels.size(0)                    # Increment the total count
    correct += (predicted == labels).sum()     # Increment the correct count
    
print('Accuracy of the network on the 10K test images: %d %%' % (100 * correct / total))

Accuracy of the network on the 10K test images: 98 %


# Save the trained FNN Model for future use

We save the trained model as a pickle that can be loaded and used later.

In [ ]:
torch.save(net.state_dict(), 'fnn_model.pkl')

# Congrats

You have done building your first Feedforward Neural Network!

In [ ]:
import os

In [35]:
os.getcwd()

'/content'

In [36]:
from google.colab import files
files.download( "/content/fnn_model.pkl" ) 

MessageError: ignored